# Loading up the data

In [2]:
import pandas as pd
import sqlite3
from collections import Counter
from sklearn.linear_model import LinearRegression, SGDClassifier,LogisticRegression, RandomizedLogisticRegression
from sklearn.metrics import classification_report, fbeta_score, accuracy_score
import numpy as np
from textblob import TextBlob
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier 

These data were run using the `make_dataset.py` script in `../src/data`

In [4]:
conn = sqlite3.connect("data/raw/data_with_fw.sqlite3")
df = pd.read_sql_query("SELECT * FROM emails WHERE folder='received'", conn)

In [5]:
# How many times did we respond, out of how many rows there are total?
len(df.query('did_reply==1')) / len(df)

0.3106553276638319

# Oracle Testing

In [4]:
y_true = df['did_reply'].values
true_indices = np.where(y_true)[0]

One of my friends went through all of the emails, and marked which ones she thought I was likely to respond to.

In [5]:
oracle_indices = [11, 31, 39, 51, 63, 68, 74, 78, 120, 122, 131, 133, 134, 138, 145, 159, 179]
y_oracle = np.zeros_like(y_true)
for i in oracle_indices:
    y_oracle[i] = 1

She was correct for four of the emails.

In [6]:
set(true_indices).intersection(oracle_indices)

{63, 131, 133, 179}

In [7]:
print(classification_report(y_true, y_oracle, target_names=['no reply', 'reply']))
print('f_2 = %s' % fbeta_score(y_true, y_oracle, 2, labels=['no reply', 'reply'], pos_label=1))

             precision    recall  f1-score   support

   no reply       0.87      0.92      0.90       172
      reply       0.24      0.14      0.18        28

avg / total       0.78      0.81      0.80       200

f_2 = 0.15503875968992248


# Baseline Testing

Can a machine doe better? Let's cut the dataset into a training slice (first 100 samples) and a validation slice (last 100 samples)

In [12]:
train_slice = slice(0, 1000)
test_slice = slice(500, 1000)


Define a feature vector based on whether the sender is somebody who we have replied to in the past.

In [7]:
favorites = list(Counter(df.iloc[train_slice].query('did_reply == 1')['sender'].values).keys())
len(favorites)

def featureExtractor(row):
    favorites_vector = [favorite in row['sender'] for favorite in favorites]
    return favorites_vector

def label(row):
    return row['did_reply']

X = np.array([featureExtractor(row) for idx, row in df.iterrows()])
y_true = np.array([label(row) for idx, row in df.iterrows()])

Machine learning magic happens here:

In [8]:
model = LinearRegression()
model.fit(X[train_slice], y_true[train_slice])
y_pred = model.predict(X[test_slice]) > .5

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [9]:
print(classification_report(y_true[test_slice], y_pred, target_names=["no reply", "reply"]))
print('f_2 = %s' % fbeta_score(y_true[test_slice], y_pred, 2, labels=['no reply', 'reply'], pos_label=1))
print(accuracy_score(y_true[test_slice], y_pred))  

             precision    recall  f1-score   support

   no reply       0.77      0.97      0.85       344
      reply       0.82      0.35      0.49       156

avg / total       0.78      0.77      0.74       500

f_2 = 0.397973950796
0.774


Additional Features

In [10]:
#Favorites extractor from before
def favoritesExtractor(row):
    favorites_vector = [favorite in row['sender'] for favorite in favorites]
    return favorites_vector

#Perform rudimentary sentiment analysis on entire message body
def sentimentBodyExtractor(row):
    if row['subject'] is not None:
        
        polarity, subjectivity = TextBlob(row['body']).sentiment
    else:
        polarity, subjectivity = 0,0
        
    return [polarity,subjectivity]

#Perform rudimentary sentiment analysis on entire message subject
def sentimentSubjectExtractor(row):
    if row['subject'] is not None:
        
        polarity, subjectivity = TextBlob(row['subject']).sentiment
    else:
        polarity, subjectivity = 0,0
        
    return [polarity,subjectivity]

#Count occurences of keywords
def keywordExtractor(row):
    keywords = ["please", "response", "request", "reply", "when", "thanks", "?", ]
    score = 0
    text = TextBlob(row['body'])
    for word in text.words:
        if str(word).lower() in keywords:
            score += 1
    return [score]


#Determine whether there exists more than one recipient
def recipientsExtractor(row):
    feature_vector = [row['multiple_recipients'] == 1]
    return feature_vector

#length of body
def lengthExtractor(row):
    return [len(row['body'])]

#Determine if "Re: " appears in subject line
def reExtractor(row):
    re_vector = [row['re_in_subject'] == 1]
    return  re_vector

#Determine if "Fwd: " appears in subject line
def fwdExtractor(row):
    fw_vector = [row['fw_in_subject'] == 1]
    return fw_vector

Run more machine learning magic, using additional features

In [11]:
X = np.array([favoritesExtractor(row)
              + lengthExtractor(row)
              + recipientsExtractor(row)
              + reExtractor(row)
              + fwdExtractor(row)
              + sentimentBodyExtractor(row)
              + sentimentSubjectExtractor(row)
              + keywordExtractor(row)
              for idx, row in df.iterrows()])
y_true = np.array([label(row) for idx, row in df.iterrows()])
print(len(X[test_slice]))
print(len(y_true[test_slice]))
print("Linear Regressor")
model = LinearRegression()
model.fit(X[train_slice], y_true[train_slice])
y_pred = model.predict(X[test_slice]) > .5
print(classification_report(y_true[test_slice], y_pred, target_names=["no reply", "reply"]))
print('f_2 = %s' % fbeta_score(y_true[test_slice], y_pred, 2, labels=['no reply', 'reply'], pos_label=1))
print(accuracy_score(y_true[test_slice], y_pred))  

500
500
Linear Regressor
             precision    recall  f1-score   support

   no reply       0.82      0.90      0.86       344
      reply       0.72      0.57      0.64       156

avg / total       0.79      0.80      0.79       500

f_2 = 0.594919786096
0.796


With multiple features, performance is about the same. Let's use a different classifier that is more suitable to the number of features:

Performance is degraded when using a new classifier. Probably for reasons of insufficient data. Lets increase the slice sizes (the more_features branch has modified the download script to download more emails):

In [142]:
train_slice = slice(0, 1500)
test_slice = slice(1500, 2000)

In [20]:
X = np.array([favoritesExtractor(row)
              + lengthExtractor(row)
              + recipientsExtractor(row)
              + reExtractor(row)
              + fwdExtractor(row)
              + sentimentBodyExtractor(row)
              + sentimentSubjectExtractor(row)
              + keywordExtractor(row)
              for idx, row in df.iterrows()])
y_true = np.array([label(row) for idx, row in df.iterrows()])
model = RandomForestClassifier()  

print("MLPClassifier")
#model = MLPClassifier(solver='relu', learning_rate='adaptive', hidden_layer_sizes=(100,100,), verbose=True, early_stopping=False, max_iter=2000)
model.fit(X[train_slice], y_true[train_slice])
#print("Features sorted by their score:")
#print(sorted(zip(map(lambda x: round(x, 4), model.feature_importances_), X[train_slice]), reverse=True))
y_pred = model.predict(X[test_slice]) > .5
print(classification_report(y_true[test_slice], y_pred, target_names=["no reply", "reply"]))
print('f_2 = %s' % fbeta_score(y_true[test_slice], y_pred, 2, labels=['no reply', 'reply'], pos_label=1))
print(accuracy_score(y_true[test_slice], y_pred))  

MLPClassifier
             precision    recall  f1-score   support

   no reply       0.97      0.99      0.98       344
      reply       0.98      0.93      0.95       156

avg / total       0.97      0.97      0.97       500

f_2 = 0.939119170984
0.972


The machine learning algorithm seems to work much better than the human.